# Type-Namespaced Functions in elastic-script

This notebook demonstrates the **namespaced function syntax** introduced in elastic-script for cleaner, more organized function calls.

## Syntax: `NAMESPACE.METHOD(args)`

Instead of `ARRAY_MAP(items, fn)`, you can now write `ARRAY.MAP(items, fn)`.

### Supported Namespaces

| Namespace | Description | Examples |
|-----------|-------------|----------|
| `ARRAY` | Array operations | `ARRAY.MAP()`, `ARRAY.FILTER()`, `ARRAY.LENGTH()` |
| `STRING` | String manipulation | `STRING.UPPER()`, `STRING.LOWER()`, `STRING.REPLACE()` |
| `NUMBER` | Numeric operations | `NUMBER.FORMAT()`, `NUMBER.ROUND()` |
| `DATE` | Date/time operations | `DATE.ADD()`, `DATE.FORMAT()`, `DATE.PARSE()` |
| `DOCUMENT` | Document operations | `DOCUMENT.KEYS()`, `DOCUMENT.VALUES()`, `DOCUMENT.MERGE()` |
| `HTTP` | HTTP requests | `HTTP.GET()`, `HTTP.POST()` |
| `K8S` | Kubernetes | `K8S.GET_PODS()`, `K8S.GET_NODES()` |
| `AWS` | AWS services | `AWS.S3_GET()`, `AWS.LAMBDA_INVOKE()` |

## Setup

First, let's set up our connection to Elasticsearch.

In [ ]:
import requests
import json
import os
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

ES_HOST = os.getenv('ES_HOST', 'https://localhost:9200')
ES_USER = os.getenv('ES_USER', 'elastic')
ES_PASSWORD = os.getenv('ES_PASSWORD', 'changeme')

def execute_escript(script):
    """Execute an elastic-script program."""
    response = requests.post(
        f"{ES_HOST}/_escript/execute",
        auth=(ES_USER, ES_PASSWORD),
        headers={'Content-Type': 'application/json'},
        json={'script': script},
        verify=False
    )
    return response.json()

print(f"Connected to: {ES_HOST}")

## Example 1: Array Operations with `ARRAY.*`

The `ARRAY` namespace provides functional programming operations on arrays.

In [ ]:
# Array operations using namespaced functions
script = '''
CREATE PROCEDURE array_demo()
BEGIN
    -- Create sample data
    DECLARE numbers ARRAY = [1, 2, 3, 4, 5];
    
    -- Get array length using ARRAY.LENGTH
    DECLARE len NUMBER;
    SET len = ARRAY.LENGTH(numbers);
    PRINT('Array length: ' || len);
    
    -- Double each number using ARRAY.MAP
    DECLARE doubled ARRAY;
    SET doubled = ARRAY.MAP(numbers, x => x * 2);
    PRINT('Doubled: ' || doubled);
    
    -- Filter even numbers using ARRAY.FILTER
    DECLARE evens ARRAY;
    SET evens = ARRAY.FILTER(doubled, x => x % 4 == 0);
    PRINT('Even (divisible by 4): ' || evens);
END PROCEDURE

CALL array_demo();
'''

result = execute_escript(script)
print(json.dumps(result, indent=2))

## Example 2: String Operations with `STRING.*`

The `STRING` namespace provides text manipulation functions.

In [ ]:
# String operations using namespaced functions
script = '''
CREATE PROCEDURE string_demo()
BEGIN
    DECLARE text STRING = 'Hello, elastic-script World!';
    
    -- Convert to uppercase
    DECLARE upper STRING;
    SET upper = STRING.UPPER(text);
    PRINT('Uppercase: ' || upper);
    
    -- Convert to lowercase
    DECLARE lower STRING;
    SET lower = STRING.LOWER(text);
    PRINT('Lowercase: ' || lower);
    
    -- Replace text
    DECLARE replaced STRING;
    SET replaced = STRING.REPLACE(text, 'World', 'Universe');
    PRINT('Replaced: ' || replaced);
END PROCEDURE

CALL string_demo();
'''

result = execute_escript(script)
print(json.dumps(result, indent=2))

## Example 3: Combined Usage with First-Class Commands

Namespaced functions work seamlessly with first-class commands like `INDEX`, `SEARCH`, etc.

In [ ]:
# Combined example: Using namespaced functions with first-class commands
script = '''
CREATE PROCEDURE data_processing()
BEGIN
    -- Sample data
    DECLARE users ARRAY = [
        {'name': 'john doe', 'email': 'john@example.com', 'active': true},
        {'name': 'jane smith', 'email': 'jane@example.com', 'active': false},
        {'name': 'bob wilson', 'email': 'bob@example.com', 'active': true}
    ];
    
    -- Transform names to uppercase using ARRAY.MAP
    DECLARE processed ARRAY;
    SET processed = ARRAY.MAP(users, user => {
        'name': STRING.UPPER(user['name']),
        'email': user['email'],
        'active': user['active']
    });
    
    PRINT('Processed users: ' || processed);
    
    -- Filter only active users
    DECLARE active_users ARRAY;
    SET active_users = ARRAY.FILTER(processed, user => user['active'] == true);
    PRINT('Active users: ' || ARRAY.LENGTH(active_users));
    
    -- Create index and store data using first-class commands
    CREATE INDEX 'users-demo' WITH MAPPINGS {
        'properties': {
            'name': {'type': 'keyword'},
            'email': {'type': 'keyword'},
            'active': {'type': 'boolean'}
        }
    };
    
    -- Index each user
    FOR user IN active_users LOOP
        INDEX user INTO 'users-demo';
    END LOOP;
    
    REFRESH 'users-demo';
    PRINT('Indexed ' || ARRAY.LENGTH(active_users) || ' active users');
END PROCEDURE

CALL data_processing();
'''

result = execute_escript(script)
print(json.dumps(result, indent=2))

## Syntax Comparison

| Old Syntax | New Namespaced Syntax |
|------------|----------------------|
| `ARRAY_MAP(arr, fn)` | `ARRAY.MAP(arr, fn)` |
| `ARRAY_FILTER(arr, fn)` | `ARRAY.FILTER(arr, fn)` |
| `ARRAY_LENGTH(arr)` | `ARRAY.LENGTH(arr)` |
| `STRING_UPPER(str)` | `STRING.UPPER(str)` |
| `STRING_REPLACE(str, old, new)` | `STRING.REPLACE(str, old, new)` |
| `DOCUMENT_KEYS(doc)` | `DOCUMENT.KEYS(doc)` |
| `DATE_ADD(date, interval)` | `DATE.ADD(date, interval)` |
| `HTTP_GET(url)` | `HTTP.GET(url)` |

Both syntaxes are supported for backward compatibility!

## Summary

Type-namespaced functions provide:

1. **Better Organization** - Functions grouped by type/domain
2. **Improved Readability** - `ARRAY.MAP` is clearer than `ARRAY_MAP`
3. **Discoverability** - Easy to find related functions
4. **Extensibility** - Easy to add new namespaces (K8S, AWS, GCP, etc.)
5. **Modern Feel** - Familiar dot-notation syntax

### Coming Soon
- `K8S.GET_PODS()`, `K8S.SCALE_DEPLOYMENT()`
- `AWS.S3_GET()`, `AWS.LAMBDA_INVOKE()`
- `GCP.PUBSUB_PUBLISH()`, `GCP.STORAGE_GET()`
- Custom namespace registration via plugins